# Earthquake ML Pipeline (Colab)
This notebook runs the full v2 pipeline: dataset generation, scaling, model training, and prediction.

**Expected data format**: each earthquake `.txt` file contains two columns: time `t` and acceleration `ag`.

If you upload files via the Colab file browser, place `.txt` files into `/content/project/earthquake_data`.


In [ ]:
# Install dependencies (Colab usually has these, but this avoids version errors).
!pip -q install -U 'tensorflow>=2.12,<3' scikit-learn


In [ ]:
import os
import numpy as np
import pickle
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GroupShuffleSplit
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

print('TensorFlow:', tf.__version__)
print('NumPy:', np.__version__)


In [ ]:
# Project structure
PROJECT_PATH = '/content/project'
EQ_FOLDER = os.path.join(PROJECT_PATH, 'earthquake_data')
DATASET_FOLDER = os.path.join(PROJECT_PATH, 'dataset_v2')
MODEL_FOLDER = os.path.join(PROJECT_PATH, 'models_v2')

os.makedirs(EQ_FOLDER, exist_ok=True)
os.makedirs(DATASET_FOLDER, exist_ok=True)
os.makedirs(MODEL_FOLDER, exist_ok=True)

print('EQ_FOLDER:', EQ_FOLDER)
print('DATASET_FOLDER:', DATASET_FOLDER)
print('MODEL_FOLDER:', MODEL_FOLDER)


In [ ]:
# Optional: upload earthquake .txt files into EQ_FOLDER
from google.colab import files
uploaded = files.upload()
for name in uploaded.keys():
    if name.lower().endswith('.txt'):
        os.replace(name, os.path.join(EQ_FOLDER, name))
print('Files in EQ_FOLDER:', os.listdir(EQ_FOLDER))


## 1) Generate dataset (v2)


In [ ]:
# CONFIG
FLOOR_LIST = [5, 10, 20, 30]
ALPHA_LIST = [0.05, 0.06, 0.065]
W_LIST = [5000, 8000, 12000, 16000]  # total building weight [kN]
SCALE_LIST = [0.5, 0.75, 1.0, 1.25, 1.5, 1.67, 1.75]

h = 0.02
g = 980.0
T0 = 4.0

# lead plug parameter (theory)
Dp = 354.0  # mm

def trapezoid(y, dx):
    # numpy compatibility: np.trapezoid is preferred, np.trapz is fallback
    return np.trapezoid(y, dx=dx) if hasattr(np, 'trapezoid') else np.trapz(y, dx=dx)

def calc_r(Dp):
    return 0.001214 * Dp + 0.5698 if Dp <= 354.0 else 1.0

def calc_rho(E_Nmm, Vp, Dp):
    r = calc_r(Dp)
    E_over_Vp = E_Nmm / Vp
    E_eff_over_Vp = r * E_over_Vp
    rho = (8.33 / 7.967) * (-0.06 + 1.25 * np.exp(-E_eff_over_Vp / 360.0))
    return min(rho, 1.0)

def extract_features(t, ag):
    dt = t[1] - t[0]
    PGA = np.max(np.abs(ag))
    PGV = np.max(np.abs(np.cumsum(ag) * dt))
    PGD = np.max(np.abs(np.cumsum(np.cumsum(ag) * dt) * dt))
    RMS = np.sqrt(np.mean(ag ** 2))
    IA = trapezoid(ag ** 2, dx=dt)
    CAV = np.sum(np.abs(ag)) * dt
    duration = t[-1] - t[0]
    return np.array([PGA, PGV, PGD, RMS, IA, CAV, duration], dtype=float)

def run_analysis(W_total, N_story, alpha_lp, t, ag, h):
    N = N_story + 1
    m = W_total / g
    omega0 = 2 * np.pi / T0

    steps = len(t)
    dt = t[1] - t[0]

    # Upper structure stiffness
    def calc_upper_structure():
        T1 = N_story / 10
        Wi = np.full(N_story, W_total / N_story)
        mi = W_total / g
        alpha = np.array([np.sum(Wi[i:]) / np.sum(Wi) for i in range(N_story)])
        Ai = 1 + ((1 / np.sqrt(alpha) - alpha) * (2 * T1) / (1 + 3 * T1))
        Aalpha = Ai * alpha
        k_prime = np.zeros((N_story, N_story))
        for i in range(N_story):
            k_prime[i, i] = Aalpha[i]
            if i < N_story - 1:
                k_prime[i, i + 1] = -Aalpha[i + 1]
        Jinv = np.eye(N_story) - np.eye(N_story, k=-1)
        M = Jinv @ k_prime
        eig = np.sort(np.real(np.linalg.eigvals(M)))
        omega = np.sqrt(eig[eig > 0])
        k1 = mi * (2 * np.pi / T1) ** 2 / omega[0] ** 2
        return k1 * Aalpha

    k_story = calc_upper_structure()

    # Find isolator stiffness kI
    omega_target = 2 * np.pi / T0

    def omega1_sq(KI):
        k = np.concatenate(([KI], k_story))
        K = np.zeros((N, N))
        for i in range(N):
            K[i, i] = k[i]
            if i < N - 1:
                K[i, i + 1] = -k[i + 1]
        Jinv = np.eye(N) - np.eye(N, k=-1)
        M = (Jinv @ K) / m
        eig = np.sort(np.real(np.linalg.eigvals(M)))
        return eig[0]

    lo, hi = 0.0, 1e6
    mid = 0.0
    for _ in range(200):
        mid = 0.5 * (lo + hi)
        if omega1_sq(mid) > omega_target ** 2:
            hi = mid
        else:
            lo = mid

    kI = mid
    kd = 10 * kI

    # Matrices
    Jinv = np.eye(N)
    for i in range(1, N):
        Jinv[i, i - 1] = -1.0
    J = np.linalg.inv(Jinv)

    K = np.zeros((N, N))
    K[0, 0] = kI
    K[0, 1] = -k_story[0]
    for i in range(1, N - 1):
        K[i, i] = k_story[i - 1]
        K[i, i + 1] = -k_story[i]
    K[-1, -1] = k_story[-1]

    C_story = (2 * h / omega0) * k_story
    C = np.zeros((N, N))
    C[0, 1] = -C_story[0]
    for i in range(1, N - 1):
        C[i, i] = C_story[i - 1]
        C[i, i + 1] = -C_story[i]
    C[-1, -1] = C_story[-1]

    # Slip model
    delta = alpha_lp * N * W_total / kd

    def slip(y, v):
        return 0.25 * v * (
            2 + np.sign(y + delta) - np.sign(y - delta)
            - np.sign(v) * (np.sign(y + delta) + np.sign(y - delta))
        )

    one = np.zeros(N)
    one[0] = 1.0

    def accel(u, v, y, agi):
        ky = np.zeros(N)
        ky[0] = kd * y[0]
        rhs = C @ v + K @ u + ky
        return -Jinv @ (rhs / m) - one * agi

    # Time integration (RK2)
    u = np.zeros((N, steps))
    v = np.zeros((N, steps))
    y = np.zeros((N, steps))

    for i in range(steps - 1):
        a1 = accel(u[:, i], v[:, i], y[:, i], ag[i])
        u2 = u[:, i] + 0.5 * dt * v[:, i]
        v2 = v[:, i] + 0.5 * dt * a1
        y2 = y[:, i] + 0.5 * dt * np.array([slip(y[0, i], v[0, i])] + [0] * (N - 1))
        a2 = accel(u2, v2, y2, ag[i])
        u[:, i + 1] = u[:, i] + dt * (v[:, i] + 2 * v2) / 3
        v[:, i + 1] = v[:, i] + dt * (a1 + 2 * a2) / 3
        y[:, i + 1] = y[:, i] + dt * (
            np.array([slip(y[0, i], v[0, i])] + [0] * (N - 1))
            + 2 * np.array([slip(y2[0], v2[0])] + [0] * (N - 1))
        ) / 3

    # Absolute acceleration
    floor_acc = np.zeros((N, steps))
    for i in range(steps):
        udd = accel(u[:, i], v[:, i], y[:, i], ag[i])
        floor_acc[:, i] = J @ udd + ag[i]

    # Max responses
    max_disp = np.zeros(30)
    max_acc = np.zeros(30)
    disp_vals = [np.max(np.abs(u[f])) for f in range(1, N)]
    acc_vals = [np.max(np.abs(floor_acc[f])) for f in range(1, N)]
    max_disp[:len(disp_vals)] = disp_vals
    max_acc[:len(acc_vals)] = acc_vals

    iso_disp_max = np.max(np.abs(u[0]))
    iso_acc_max = np.max(np.abs(floor_acc[0]))

    # Energy E
    E = 0.0
    for i in range(steps - 1):
        du = u[0, i + 1] - u[0, i]
        Q1 = kI * u[0, i] + kd * y[0, i]
        Q2 = kI * u[0, i + 1] + kd * y[0, i + 1]
        E += 0.5 * (Q1 + Q2) * du

    # Output vector for ML (63)
    Y_ml = np.concatenate([
        max_disp,
        max_acc,
        [iso_disp_max],
        [iso_acc_max],
        [E],
    ])
    return Y_ml

# Dataset generation
X_list, Y_list, groups_list = [], [], []

for file in os.listdir(EQ_FOLDER):
    if not file.endswith('.txt'):
        continue
    eq_path = os.path.join(EQ_FOLDER, file)
    t, ag0 = np.loadtxt(eq_path, unpack=True)

    for scale in SCALE_LIST:
        ag = ag0 * scale
        features = extract_features(t, ag)

        for W in W_LIST:
            for floors in FLOOR_LIST:
                for alpha in ALPHA_LIST:
                    X = np.concatenate([[W, floors, h, alpha, scale], features])
                    Y = run_analysis(W, floors, alpha, t, ag, h)

                    X_list.append(X)
                    Y_list.append(Y)
                    groups_list.append(file)

        print(f'Done EQ: {file} (scale={scale})')

X = np.array(X_list, dtype=float)
Y = np.array(Y_list, dtype=float)
groups = np.array(groups_list, dtype=object)

np.save(os.path.join(DATASET_FOLDER, 'X.npy'), X)
np.save(os.path.join(DATASET_FOLDER, 'Y_ml.npy'), Y)
np.save(os.path.join(DATASET_FOLDER, 'groups.npy'), groups)

print('Dataset v2 generation complete.')
print('X shape =', X.shape)
print('Y_ml shape =', Y.shape)
print('groups shape =', groups.shape)


## 2) Scale dataset (v2)


In [ ]:
X_path = os.path.join(DATASET_FOLDER, 'X.npy')
Y_path = os.path.join(DATASET_FOLDER, 'Y_ml.npy')
G_path = os.path.join(DATASET_FOLDER, 'groups.npy')

print('Loading dataset_v2...')
X = np.load(X_path)
Y = np.load(Y_path)
groups = np.load(G_path, allow_pickle=True)

assert X.shape[0] == Y.shape[0] == groups.shape[0], 'Sample count mismatch!'

print('X shape:', X.shape)
print('Y shape:', Y.shape)
print('groups shape:', groups.shape)

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

print('Fitting scalers...')
X_scaled = x_scaler.fit_transform(X)
Y_scaled = y_scaler.fit_transform(Y)

np.save(os.path.join(DATASET_FOLDER, 'X_scaled.npy'), X_scaled)
np.save(os.path.join(DATASET_FOLDER, 'Y_scaled.npy'), Y_scaled)

with open(os.path.join(DATASET_FOLDER, 'x_scaler.pkl'), 'wb') as f:
    pickle.dump(x_scaler, f)
with open(os.path.join(DATASET_FOLDER, 'y_scaler.pkl'), 'wb') as f:
    pickle.dump(y_scaler, f)

print('Saved:')
print(' - X_scaled.npy, Y_scaled.npy')
print(' - x_scaler.pkl, y_scaler.pkl')
print('[02_scale_dataset_v2 DONE]')


## 3) Train models (v2)


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

X = np.load(os.path.join(DATASET_FOLDER, 'X_scaled.npy'))
Y = np.load(os.path.join(DATASET_FOLDER, 'Y_scaled.npy'))
groups = np.load(os.path.join(DATASET_FOLDER, 'groups.npy'), allow_pickle=True)

input_dim = X.shape[1]
output_dim = Y.shape[1]

print('X shape=', X.shape, 'Y shape=', Y.shape, 'output_dim=', output_dim)

def build_model(input_dim, output_dim):
    inp = layers.Input(shape=(input_dim,))
    x = layers.Dense(128)(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.15)(x)

    x = layers.Dense(256)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.15)(x)

    x = layers.Dense(128)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    out = layers.Dense(output_dim, activation='linear')(x)
    model = models.Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss=tf.keras.losses.Huber(delta=1.0),
        metrics=[tf.keras.metrics.MeanAbsoluteError()]
    )
    return model

def train_one_floor(floor_value):
    # Filter by raw floors (before scaling)
    X_raw = np.load(os.path.join(DATASET_FOLDER, 'X.npy'))
    floors_raw = X_raw[:, 1].astype(int)
    idx = np.where(floors_raw == floor_value)[0]

    Xf = X[idx]
    Yf = Y[idx]
    gf = groups[idx]

    print(f'
=== Training for floors={floor_value} ===')
    print('samples =', len(idx))

    splitter = GroupShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
    train_idx, val_idx = next(splitter.split(Xf, Yf, groups=gf))

    X_train, Y_train = Xf[train_idx], Yf[train_idx]
    X_val, Y_val = Xf[val_idx], Yf[val_idx]

    model = build_model(input_dim, output_dim)
    ckpt_path = os.path.join(MODEL_FOLDER, f'model_f{floor_value}.keras')

    callbacks = [
        ModelCheckpoint(ckpt_path, monitor='val_loss', save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-5),
        EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True),
    ]

    model.fit(
        X_train, Y_train,
        validation_data=(X_val, Y_val),
        epochs=400,
        batch_size=32,
        callbacks=callbacks,
        verbose=1,
    )

    final_path = os.path.join(MODEL_FOLDER, f'model_f{floor_value}_final.keras')
    model.save(final_path)
    print('Saved best: ', ckpt_path)
    print('Saved final:', final_path)

for f in [5, 10, 20, 30]:
    train_one_floor(f)

print('All models trained.')


## 4) Predict (v2)
Update the parameters in the cell below, then run.


In [ ]:
# User inputs
W_total = 8437.5
floors = 10  # 5, 10, 20, 30
h = 0.02
alpha = 0.05
scale = 1.0
eq_path = os.path.join(EQ_FOLDER, 'hoei-kobe.txt')  # change to your file

# Load scalers
with open(os.path.join(DATASET_FOLDER, 'x_scaler.pkl'), 'rb') as f:
    x_scaler = pickle.load(f)
with open(os.path.join(DATASET_FOLDER, 'y_scaler.pkl'), 'rb') as f:
    y_scaler = pickle.load(f)

Dp = 354.0
H = 523.0
g = 980.0

def trapezoid(y, dx):
    return np.trapezoid(y, dx=dx) if hasattr(np, 'trapezoid') else np.trapz(y, dx=dx)

def calc_r(Dp):
    return 0.001214 * Dp + 0.5698 if Dp <= 354.0 else 1.0

def calc_rho(E_Nmm, Vp, Dp):
    r = calc_r(Dp)
    E_over_Vp = E_Nmm / Vp
    E_eff_over_Vp = r * E_over_Vp
    rho = (8.33 / 7.967) * (-0.06 + 1.25 * np.exp(-E_eff_over_Vp / 360.0))
    return min(rho, 1.0)

def extract_features(t, ag):
    dt = t[1] - t[0]
    PGA = np.max(np.abs(ag))
    PGV = np.max(np.abs(np.cumsum(ag) * dt))
    PGD = np.max(np.abs(np.cumsum(np.cumsum(ag) * dt) * dt))
    RMS = np.sqrt(np.mean(ag ** 2))
    IA = trapezoid(ag ** 2, dx=dt)
    CAV = np.sum(np.abs(ag)) * dt
    duration = t[-1] - t[0]
    return np.array([PGA, PGV, PGD, RMS, IA, CAV, duration], dtype=float)

if floors not in [5, 10, 20, 30]:
    raise ValueError('floors must be 5, 10, 20, or 30')
if not os.path.exists(eq_path):
    raise FileNotFoundError('Earthquake file not found: ' + eq_path)

t, ag0 = np.loadtxt(eq_path, unpack=True)
ag = ag0 * scale

features = extract_features(t, ag)
X = np.concatenate([[W_total, floors, h, alpha, scale], features]).reshape(1, -1)
X_scaled = x_scaler.transform(X)

model_path = os.path.join(MODEL_FOLDER, f'model_f{floors}.keras')
model = tf.keras.models.load_model(model_path, compile=False)
print('Loaded model:', model_path)

Y_scaled = model.predict(X_scaled, verbose=0)
Y = y_scaler.inverse_transform(Y_scaled)[0]

max_disp = Y[0:30]
max_acc  = Y[30:60]
iso_disp = Y[60]
iso_acc  = Y[61]
E        = Y[62]

SigmaW = W_total * 1000.0
Vp = alpha * SigmaW * H / 8.33
E_Nmm = E * 1e4
Evp = E_Nmm / Vp
rho = calc_rho(E_Nmm, Vp, Dp)

print('\n=== ML PREDICTION RESULT (v2) ===')
print('\n--- Floor Maximum Displacement [cm] ---')
for i in range(floors):
    print(f'Floor {i+1:2d}: {max_disp[i]:.5f}')

print('\n--- Floor Maximum Absolute Acceleration [cm/s^2] ---')
for i in range(floors):
    print(f'Floor {i+1:2d}: {max_acc[i]:.5f}')

print('\n--- Isolator Response ---')
print(f'Isolator max displacement = {iso_disp:.5f} cm')
print(f'Isolator max abs accel    = {iso_acc:.5f} cm/s^2')

print('\n--- Energy & Degradation Indices (Hybrid Physics) ---')
print(f'E      = {E:.6e} kN*cm')
print(f'Vp     = {Vp:.6e} (computed)')
print(f'E / Vp = {Evp:.6e} (computed)')
print(f'rho    = {rho:.6f} (computed)')

print('\nPrediction complete.')
